In [ ]:
import os
import numpy as np
import librosa
import soundfile as sf
from sklearn.decomposition import NMF
from tqdm import tqdm
import matplotlib.pyplot as plt
import librosa.display

In [ ]:
# Path and Suffix related Variables
start = "IDMT-SMT-DRUMS-V2"
annotation = "annotation_xml"
audio = "audio"
mix_wav ="MIX.wav"
train_wav = "train.wav"
annotation_files = os.listdir(os.path.join(start,annotation))
drum_files = os.listdir(os.path.join(start,audio))
drum_names = [i.split("#")[0] for i in annotation_files]

In [ ]:
def extract_spectrogram(data_path):
    audio, sr = librosa.load(data_path, sr=None)
    print(sr)
    spectrogram = librosa.stft(audio, n_fft=2048, hop_length=256, center=False, win_length=2048)
    M = abs(spectrogram)
    phase= spectrogram/(M+2.2204e-16)
    M_db = librosa.amplitude_to_db(M)
    plt.figure()
    librosa.display.specshow(M_db, hop_length=256,y_axis='linear',sr = 44100, x_axis='time')
    plt.show()
    return M, phase

In [ ]:
def perform_NMF_source(data, min_length, n_components=300):
    #Spectrogram
    #n_component default 300
    M = data['M']
    new_M = M[:,:min_length]
    
    #beta_loos for KL divergence
    #Solver Mu to use multiplicative update
    model = NMF(n_components=n_components, beta_loss=1, init='random', solver='mu', max_iter=600, random_state=0)
    
    B = model.fit_transform(new_M)
    
    # M =B*H
    data['base'] =B
    data['M'] = new_M
    data['model'] = model

In [ ]:
def seperate_signals(M_mix, base, n_sources, n_iter):
    pq = np.ones(M_mix.shape)    
    B =base
    
    W = np.random.rand(B.shape[1],M_mix.shape[1])
    for _ in tqdm(range(n_iter)):
        diff = M_mix/np.dot(B,W)
        W_numer = np.dot(B.T, diff)
        W_denom = np.dot(B.T, pq)
        W = np.multiply(W, W_numer/W_denom)
        #Replace 0 with very small number to avoid zero devision
        W[W==0]=0.0001

    seperated_sources = []
    
    for i in range(0,n_sources):
        data = {"W": W[i*300:(i+1)*300,:],
                "B": B[:,i*300:(i+1)*300]}
        seperated_sources.append(data)

    return seperated_sources

In [ ]:
test = drum_names[0]
print("For Example we are going to use", test)
test_files= [i for i in drum_files if i.startswith(test)]
source_files = []
for file in test_files:
    if file.endswith(mix_wav):
        mix_wav = file
    elif file.endswith(train_wav):
        source_files.append(file)

In [ ]:
m_phase_infos = []
lengths = []
for source in source_files:
    source_path = os.path.join(start,audio,source)
    M,phase = extract_spectrogram(source_path)
    lengths.append(M.shape[1])

    data = {'name': source,
            'M': M,
            'phase':phase,
            'base': None,
            'model':None}
    m_phase_infos.append(data)
min_length = min(lengths)

In [ ]:
for i in m_phase_infos:
    perform_NMF_source(i,min_length)

In [ ]:
bases = [i['base'] for i in m_phase_infos]
base = np.concatenate(bases, axis=1)

In [ ]:
mix_path = os.path.join(start,audio,mix_wav)
M_m,phase_m = extract_spectrogram(mix_path)

In [ ]:
seperated_sources=seperate_signals(M_m, base, 3, 500)

In [ ]:
for index, signal in enumerate(seperated_sources):
    isExist = os.path.exists("output")
    if not isExist:
        os.mkdir("output")
    H = signal['W']
    B = signal['B']
    H[H==0.0001] = 0
    M_test = np.dot(B,H)
    signal_hat = librosa.istft(M_test*phase_m, hop_length=256, center=False, win_length=1024)
    sf.write(f"output/test_{index}.wav", signal_hat, samplerate=44100)
    extract_spectrogram(f"output/test_{index}.wav")